<a href="https://colab.research.google.com/github/Tensai-Pro/SRI-PracticeTasks/blob/master/Word%20Embeddings/3)%20fastText.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [78]:
!pip install fasttext

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [79]:
import pandas as pd
import numpy as np

from gensim.utils import simple_preprocess
from gensim.models import FastText

import nltk;
nltk.download('stopwords');
from nltk.corpus import stopwords;

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier

import fasttext

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [80]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [81]:
df = pd.read_csv('/content/gdrive/MyDrive/Practice Files/train.zip')
df.head(3)

,Class Index,Title,Description
0,3,Wall St. Bears Claw Back Into the Black (Reuters),"Reuters - Short-sellers, Wall Street's dwindli..."
1,3,Carlyle Looks Toward Commercial Aerospace (Reu...,Reuters - Private investment firm Carlyle Grou...
2,3,Oil and Economy Cloud Stocks' Outlook (Reuters),Reuters - Soaring crude prices plus worries\ab...


# Предобработка и подготовка для fastText

In [82]:
stopwords = stopwords.words('english')

In [83]:
def prepare_text(text):
  tokens = simple_preprocess(text)
  tokens = [tok for tok in tokens if tok not in stopwords]
  return ' '.join(tokens)

In [84]:
df['clean_text'] = df.Description.apply(prepare_text)
df.head()

,Class Index,Title,Description,clean_text
0,3,Wall St. Bears Claw Back Into the Black (Reuters),"Reuters - Short-sellers, Wall Street's dwindli...",reuters short sellers wall street dwindling ba...
1,3,Carlyle Looks Toward Commercial Aerospace (Reu...,Reuters - Private investment firm Carlyle Grou...,reuters private investment firm carlyle group ...
2,3,Oil and Economy Cloud Stocks' Outlook (Reuters),Reuters - Soaring crude prices plus worries\ab...,reuters soaring crude prices plus worries econ...
3,3,Iraq Halts Oil Exports from Main Southern Pipe...,Reuters - Authorities have halted oil export\f...,reuters authorities halted oil export flows ma...
4,3,"Oil prices soar to all-time record, posing new...","AFP - Tearaway world oil prices, toppling reco...",afp tearaway world oil prices toppling records...


In [85]:
X_train, X_test, y_train, y_test = train_test_split(df.clean_text, df['Class Index'],
                                                    shuffle=True, random_state=42)

In [86]:
with open('train_data.txt', 'w', encoding='utf-8') as f:
  for i in range(len(X_train)):
    f.write('__label__' + str(y_train.values[i]) + ' ' + X_train.values[i] + '\n')

with open('test_data.txt', 'w', encoding='utf-8') as f:
  for i in range(len(X_test)):
    f.write('__label__' + str(y_test.values[i]) + ' ' + X_test.values[i] + '\n')

# fastText

## Библиотека fastTest

In [ ]:
model = fasttext.train_supervised(input='train_data.txt', epoch=50, wordNgrams=2,
                                  lr=0.005)

In [ ]:
result = model.test('test_data.txt')

In [ ]:
print('Test Accuracy:', result)

Test Accuracy: (30000, 0.9055333333333333, 0.9055333333333333)


In [ ]:
model.predict(X_test.values[10])

(('__label__4',), array([0.92115545]))

In [ ]:
y_test.values[10]

4

## Модуль gensim fastText

In [87]:
clean_text_tokens = X_train.apply(lambda x: x.split(' '))

In [88]:
model_gensim = FastText(clean_text_tokens, vector_size=100, window=5,
                        min_count=5, workers=4)

In [91]:
model_gensim.train(corpus_iterable=clean_text_tokens, 
                   total_examples=len(clean_text_tokens), epochs=10)

(17241581, 18136920)

In [92]:
model_gensim.wv['gastroenteritis']

array([-0.72896016,  0.07606808,  0.36694753, -0.07568449,  0.07119192,
       -1.0664583 ,  0.378626  , -0.6597585 ,  1.0551571 , -0.21986134,
        0.38397348, -0.5202483 , -0.03893563, -0.37454224, -0.30293787,
        0.01861934, -0.32655925, -0.31563473,  0.04078551,  0.11752435,
       -0.05610346, -0.2316435 , -0.4092508 ,  0.7134848 , -0.2200847 ,
       -0.12685187,  1.2535815 ,  0.29173735,  0.73042035,  0.43831882,
        0.3306061 , -0.6142112 ,  0.32873783, -0.039017  ,  0.04482775,
        0.11474589, -0.21255836, -0.26040208, -0.19691506, -1.4328423 ,
       -0.05345409,  1.1338176 ,  1.3801588 ,  0.47705725,  0.75716144,
        0.3886566 ,  0.29731846, -0.38727447,  0.629503  ,  0.21525204,
       -0.6172844 ,  0.49279812, -0.12916063,  0.624795  , -0.44863018,
       -0.48579386,  0.2085447 , -0.28596294,  1.359474  ,  0.4554377 ,
       -0.84833   ,  0.3223625 ,  0.5812951 , -0.31436485,  0.4969337 ,
        0.67428404, -0.99212873, -0.3689961 ,  0.70073247, -0.25

In [93]:
model_gensim.wv.most_similar('castle')

[('newcastle', 0.7588932514190674),
 ('nestle', 0.6795447468757629),
 ('winchester', 0.6432747840881348),
 ('hustle', 0.6301568746566772),
 ('manchester', 0.622786283493042),
 ('castillo', 0.604910671710968),
 ('rochester', 0.6043378710746765),
 ('chester', 0.5936146378517151),
 ('cast', 0.5867555141448975),
 ('castilla', 0.5698596239089966)]

# Векторизация

In [94]:
def vectorize(text):
    vectorized_text = np.array([model_gensim.wv[word] for word in text if word in model_gensim.wv])
    if len(vectorized_text) == 0:
        return np.zeros(100)
    return vectorized_text.mean(axis=0)

In [95]:
X_train_vectorized = np.array([vectorize(sentence) for sentence in X_train])
X_test_vectorized = np.array([vectorize(sentence) for sentence in X_test])

In [96]:
X_train_vectorized.shape

(90000, 100)

# Классификация

In [97]:
scaler = MinMaxScaler()

In [98]:
X_train_scaled = scaler.fit_transform(X_train_vectorized)
X_test_scaled = scaler.transform(X_test_vectorized)

In [102]:
clf_nb = MultinomialNB().fit(X_train_scaled, y_train)

In [103]:
y_pred_nb = clf_nb.predict(X_test_scaled)

In [104]:
acc_score = clf_nb.score(X_test_scaled, y_test) * 100
print(f'Accuracy: {acc_score}%')

Accuracy: 26.96%
